# KAMPOENG KREASI EXPERIMENTAL AI

# **UNTUK UJI COBA, SILAHKAN SETELAH LAKUKAN RUN ALL LANGSUNG MENUJU CELL PALING BAWAH**

In [2]:
!git clone https://github.com/MarcoAlandAdinanda/kampoengkreasi.git
from tensorflow.keras.models import load_model

fatal: destination path 'kampoengkreasi' already exists and is not an empty directory.


In [ ]:
intents = json.loads(open('/content/kampoengkreasi/kampoengkreasi_intents.json').read())
words = pickle.load(open('/content/kampoengkreasi/words.pkl', 'rb'))
classes = pickle.load(open('/content/kampoengkreasi/classes.pkl', 'rb'))

In [ ]:
model = load_model('/content/kampoengkreasi/kampoengkreasiv1.h5')

In [ ]:
def clean_up_sentence(sentence):
  sentence_words = nltk.word_tokenize(sentence)
  return sentence_words

def bag_of_words(sentence):
  sentence_words = clean_up_sentence(sentence)
  bag = [0] * len(words)
  for w in sentence_words:
    for i, word in enumerate(words):
      if word == w:
        bag[i] = 1
  return np.array(bag)

def predict_class(sentence):
  bow = bag_of_words(sentence)
  res = model.predict(np.array([bow]))[0]
  ERROR_THRESHOLD = 0.25
  results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]

  results.sort(key=lambda x: x[1], reverse=True)
  return_list = []
  for r in results:
    return_list.append({'intent': classes[r[0]], 'probability': str(r[1])})
  return return_list

def format_address(address):
    address_lines = address.splitlines()
    formatted_address = ' '.join(address_lines)
    return formatted_address

def get_response(intents_list, intents_json):
    tag = intents_list[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if i['tag'] == tag:
            result = random.choice(i['responses'])
            result = result.replace(", ", ",\n")
            if tag not in ['salam', 'perkenalan', 'terima_kasih', 'opsi', 'pencipta', 'noinput']:
                if 'Alamat:' in result:
                    address_start = result.index('Alamat:')
                    address_end = result.index('Indonesia,') + len('Indonesia,')
                    formatted_address = format_address(result[address_start:address_end])
                    result = result[:address_start] + formatted_address + result[address_end:]
                result = f"Tentu saja!\n{result}\nSemoga sesuai :)\nApakah ada yang ingin ditanyakan lagi?"
            break
    return result

In [ ]:
while True:
  message = input("")
  ints = predict_class(message)
  res = get_response(ints, intents)
  print(res)
  if message == "berhenti":
    break